# GOAL 2 - PREDICT POSITION

In [144]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


# Import packages

In [158]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    # LabelEncoder,
)
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
)

# Load dataset

In [147]:
df_final = pd.read_csv("project_df_final.csv", delimiter=",")

In [148]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2078713 entries, 0 to 2078712
Data columns (total 30 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   local_timestamp           object 
 1   headpose_pitch            float64
 2   headpose_roll             float64
 3   headpose_yaw              float64
 4   looking_at_screen         int64  
 5   person_id                 object 
 6   poi_id                    int64  
 7   x_world                   float64
 8   y_world                   float64
 9   z_world                   float64
 10  present_in_1              int64  
 11  present_in_2              int64  
 12  present_in_5              int64  
 13  gender                    object 
 14  age                       float64
 15  total_time_spent_seconds  int64  
 16  cumulative_time           int64  
 17  dx_world                  float64
 18  dy_world                  float64
 19  d_time                    float64
 20  x_velocity              

## Add labels to dataframe for modeling

In [149]:
def add_labels_in_n(d, n):
    x_col = "x_in_" + str(n)
    y_col = "y_in_" + str(n)
    person_col = "person_in_" + str(n)

    d[x_col] = d["x_world"].shift(-n)
    d[y_col] = d["y_world"].shift(-n)
    d[person_col] = d["person_id"].shift(-n)

    return d[d["person_id"] == d[person_col]]

## Create subdataframes per predictive window

In [150]:
df_model_5 = add_labels_in_n(df_final, 5)
# df_model_5[["person_id", "person_in_5", "x_world", "x_in_5", "y_world", "y_in_5"]].head(40)

df_model_2 = add_labels_in_n(df_final, 2)
# df_model_2[["person_id", "person_in_2", "x_world", "x_in_2", "y_world", "y_in_2"]].head(40)

df_model_1 = add_labels_in_n(df_final, 1)
# df_model_1[["person_id", "person_in_1", "x_world", "x_in_1", "y_world", "y_in_1"]].head(40)

# Data Modeling

## Random Forest

### Define functions to create subdataframes per installation (POI) and predictive time window (1,2,5 seconds)

In [151]:
def define_target_and_df_per_window_5(
    sec=5, df=df_model_5, columns=[],
):
    if columns == []:
        columns = [
            "poi_id",
            "person_id",
            "x_world",
            "y_world",
            "age",
            "gender",
            "looking_at_screen",
            "ema_5_x",
            "ema_5_y",
            "ema_5_velocity",
            "cumulative_time",
            "dx_world",
            "dy_world",
            "d_time",
            "x_velocity",
            "y_velocity",
            "velocity",
            "distance_to_camera",
            "ema_10_x",
            "ema_10_y",
            "ema_10_velocity",
            "x_in_5",
            "y_in_5",
        ]

    df_model = df[columns]
    print("The model target window is " + str(sec) + " seconds. \n")
    print(df_model.info())

    df_inst_1 = df_model.loc[df["poi_id"] == 444]
    df_inst_450 = df_model.loc[df["poi_id"] == 450]
    df_inst_453 = df_model.loc[df["poi_id"] == 453]
    df_inst_456 = df_model.loc[df["poi_id"] == 456]
    df_inst_457 = df_model.loc[df["poi_id"] == 457]
    df_inst_466 = df_model.loc[df["poi_id"] == 466]
    df_inst_529 = df_model.loc[df["poi_id"] == 529]

    return (
        df_inst_1,
        df_inst_450,
        df_inst_453,
        df_inst_456,
        df_inst_457,
        df_inst_466,
        df_inst_529,
    )

In [152]:
def define_target_and_df_per_window_2(
    sec=2, df=df_model_2, columns=[],
):
    if columns == []:
        columns = [
            "poi_id",
            "person_id",
            "x_world",
            "y_world",
            "age",
            "gender",
            "looking_at_screen",
            "ema_5_x",
            "ema_5_y",
            "ema_5_velocity",
            "cumulative_time",
            "dx_world",
            "dy_world",
            "d_time",
            "x_velocity",
            "y_velocity",
            "velocity",
            "distance_to_camera",
            "ema_10_x",
            "ema_10_y",
            "ema_10_velocity",
            "x_in_2",
            "y_in_2",
        ]

    df_model = df[columns]
    print("The model target window is " + str(sec) + " seconds. \n")
    print(df_model.info())

    df_inst_1 = df_model.loc[df["poi_id"] == 444]
    df_inst_450 = df_model.loc[df["poi_id"] == 450]
    df_inst_453 = df_model.loc[df["poi_id"] == 453]
    df_inst_456 = df_model.loc[df["poi_id"] == 456]
    df_inst_457 = df_model.loc[df["poi_id"] == 457]
    df_inst_466 = df_model.loc[df["poi_id"] == 466]
    df_inst_529 = df_model.loc[df["poi_id"] == 529]

    return (
        df_inst_1,
        df_inst_450,
        df_inst_453,
        df_inst_456,
        df_inst_457,
        df_inst_466,
        df_inst_529,
    )

In [153]:
def define_target_and_df_per_window_1(
    sec=1, df=df_model_1, columns=[],
):
    if columns == []:
        columns = [
            "poi_id",
            "person_id",
            "x_world",
            "y_world",
            "age",
            "gender",
            "looking_at_screen",
            "ema_5_x",
            "ema_5_y",
            "ema_5_velocity",
            "cumulative_time",
            "dx_world",
            "dy_world",
            "d_time",
            "x_velocity",
            "y_velocity",
            "velocity",
            "distance_to_camera",
            "ema_10_x",
            "ema_10_y",
            "ema_10_velocity",
            "x_in_1",
            "y_in_1",
        ]

    df_model = df[columns]
    print("The model target window is " + str(sec) + " seconds. \n")
    print(df_model.info())

    df_inst_1 = df_model.loc[df["poi_id"] == 444]
    df_inst_450 = df_model.loc[df["poi_id"] == 450]
    df_inst_453 = df_model.loc[df["poi_id"] == 453]
    df_inst_456 = df_model.loc[df["poi_id"] == 456]
    df_inst_457 = df_model.loc[df["poi_id"] == 457]
    df_inst_466 = df_model.loc[df["poi_id"] == 466]
    df_inst_529 = df_model.loc[df["poi_id"] == 529]

    return (
        df_inst_1,
        df_inst_450,
        df_inst_453,
        df_inst_456,
        df_inst_457,
        df_inst_466,
        df_inst_529,
    )

### Define function to train-test split while keeping the grouping per POI and person ID

In [154]:
def split_per_group(d):

    train_inds, test_inds = next(
        GroupShuffleSplit(test_size=0.2, n_splits=2, random_state=42).split(
            d, groups=d["person_id"]
        )
    )

    train = d.iloc[train_inds]
    test = d.iloc[test_inds]

    y_train = train.iloc[:, -2:]
    y_test = test.iloc[:, -2:]

    # Drop person_id and POI_id from our POI dataframe, as well as the targets x_in_n and y_in_n
    X_train = train[train.columns[2:-2]]
    X_test = test[test.columns[2:-2]]

    return X_train, y_train, X_test, y_test


(
    df_inst_444_1,
    df_inst_450_1,
    df_inst_453_1,
    df_inst_456_1,
    df_inst_457_1,
    df_inst_466_1,
    df_inst_529_1,
) = define_target_and_df_per_window_1(1,)

X_train_444_1, y_train_444_1, X_test_444_1, y_test_444_1 = split_per_group(
    df_inst_444_1
)
X_train_450_1, y_train_450_1, X_test_450_1, y_test_450_1 = split_per_group(
    df_inst_450_1
)
X_train_453_1, y_train_453_1, X_test_453_1, y_test_453_1 = split_per_group(
    df_inst_453_1
)
X_train_456_1, y_train_456_1, X_test_456_1, y_test_456_1 = split_per_group(
    df_inst_456_1
)
X_train_457_1, y_train_457_1, X_test_457_1, y_test_457_1 = split_per_group(
    df_inst_457_1
)
X_train_466_1, y_train_466_1, X_test_466_1, y_test_466_1 = split_per_group(
    df_inst_466_1
)
X_train_529_1, y_train_529_1, X_test_529_1, y_test_529_1 = split_per_group(
    df_inst_529_1
)

(
    df_inst_444_2,
    df_inst_450_2,
    df_inst_453_2,
    df_inst_456_2,
    df_inst_457_2,
    df_inst_466_2,
    df_inst_529_2,
) = define_target_and_df_per_window_2(2,)

X_train_444_2, y_train_444_2, X_test_444_2, y_test_444_2 = split_per_group(
    df_inst_444_2
)
X_train_450_2, y_train_450_2, X_test_450_2, y_test_450_2 = split_per_group(
    df_inst_450_2
)
X_train_453_2, y_train_453_2, X_test_453_2, y_test_453_2 = split_per_group(
    df_inst_453_2
)
X_train_456_2, y_train_456_2, X_test_456_2, y_test_456_2 = split_per_group(
    df_inst_456_2
)
X_train_457_2, y_train_457_2, X_test_457_2, y_test_457_2 = split_per_group(
    df_inst_457_2
)
X_train_466_2, y_train_466_2, X_test_466_2, y_test_466_2 = split_per_group(
    df_inst_466_2
)
X_train_529_2, y_train_529_2, X_test_529_2, y_test_529_2 = split_per_group(
    df_inst_529_2
)


(
    df_inst_444_5,
    df_inst_450_5,
    df_inst_453_5,
    df_inst_456_5,
    df_inst_457_5,
    df_inst_466_5,
    df_inst_529_5,
) = define_target_and_df_per_window_5(5,)

X_train_444_5, y_train_444_5, X_test_444_5, y_test_444_5 = split_per_group(
    df_inst_444_5
)
X_train_450_5, y_train_450_5, X_test_450_5, y_test_450_5 = split_per_group(
    df_inst_450_5
)
X_train_453_5, y_train_453_5, X_test_453_5, y_test_453_5 = split_per_group(
    df_inst_453_5
)
X_train_456_5, y_train_456_5, X_test_456_5, y_test_456_5 = split_per_group(
    df_inst_456_5
)
X_train_457_5, y_train_457_5, X_test_457_5, y_test_457_5 = split_per_group(
    df_inst_457_5
)
X_train_466_5, y_train_466_5, X_test_466_5, y_test_466_5 = split_per_group(
    df_inst_466_5
)
X_train_529_5, y_train_529_5, X_test_529_5, y_test_529_5 = split_per_group(
    df_inst_529_5
)

train_test_per_camera_dict = {
    "cam444_1": [X_train_444_1, y_train_444_1, X_test_444_1, y_test_444_1],
    "cam450_1": [X_train_450_1, y_train_450_1, X_test_450_1, y_test_450_1],
    "cam453_1": [X_train_453_1, y_train_453_1, X_test_453_1, y_test_453_1],
    "cam456_1": [X_train_456_1, y_train_456_1, X_test_456_1, y_test_456_1],
    "cam457_1": [X_train_457_1, y_train_457_1, X_test_457_1, y_test_457_1],
    "cam466_1": [X_train_466_1, y_train_466_1, X_test_466_1, y_test_466_1],
    "cam529_1": [X_train_529_1, y_train_529_1, X_test_529_1, y_test_529_1],
    "cam444_2": [X_train_444_2, y_train_444_2, X_test_444_2, y_test_444_2],
    "cam450_2": [X_train_450_2, y_train_450_2, X_test_450_2, y_test_450_2],
    "cam453_2": [X_train_453_2, y_train_453_2, X_test_453_2, y_test_453_2],
    "cam456_2": [X_train_456_2, y_train_456_2, X_test_456_2, y_test_456_2],
    "cam457_2": [X_train_457_2, y_train_457_2, X_test_457_2, y_test_457_2],
    "cam466_2": [X_train_466_2, y_train_466_2, X_test_466_2, y_test_466_2],
    "cam529_2": [X_train_529_2, y_train_529_2, X_test_529_2, y_test_529_2],
    "cam444_5": [X_train_444_5, y_train_444_5, X_test_444_5, y_test_444_5],
    "cam450_5": [X_train_450_5, y_train_450_5, X_test_450_5, y_test_450_5],
    "cam453_5": [X_train_453_5, y_train_453_5, X_test_453_5, y_test_453_5],
    "cam456_5": [X_train_456_5, y_train_456_5, X_test_456_5, y_test_456_5],
    "cam457_5": [X_train_457_5, y_train_457_5, X_test_457_5, y_test_457_5],
    "cam466_5": [X_train_466_5, y_train_466_5, X_test_466_5, y_test_466_5],
    "cam529_5": [X_train_529_5, y_train_529_5, X_test_529_5, y_test_529_5],
}

The model target window is 1 seconds. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1986463 entries, 0 to 2078711
Data columns (total 23 columns):
 #   Column              Dtype  
---  ------              -----  
 0   poi_id              int64  
 1   person_id           object 
 2   x_world             float64
 3   y_world             float64
 4   age                 float64
 5   gender              object 
 6   looking_at_screen   int64  
 7   ema_5_x             float64
 8   ema_5_y             float64
 9   ema_5_velocity      float64
 10  cumulative_time     int64  
 11  dx_world            float64
 12  dy_world            float64
 13  d_time              float64
 14  x_velocity          float64
 15  y_velocity          float64
 16  velocity            float64
 17  distance_to_camera  float64
 18  ema_10_x            float64
 19  ema_10_y            float64
 20  ema_10_velocity     float64
 21  x_in_1              float64
 22  y_in_1              float64
dtypes: float64(18), i

### Define Random Forest Regression Model Pipeline

In [155]:
# Define numerical features
numerical_features = [
    "x_world",
    "y_world",
    "age",
    "ema_5_x",
    "ema_5_y",
    "ema_5_velocity",
    "cumulative_time",
    "dx_world",
    "dy_world",
    "d_time",
    "x_velocity",
    "y_velocity",
    "velocity",
    "distance_to_camera",
    "ema_10_x",
    "ema_10_y",
    "ema_10_velocity",
]

# Define categorical features
categorical_features = ["gender", "looking_at_screen"]

# Define transformers
numerical_transformer = Pipeline(steps=[("scaler", StandardScaler()),])

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
)

# Combine feature transformers
data_transformer = ColumnTransformer(
    transformers=[
        ("numerical", numerical_transformer, numerical_features),
        ("categorical", categorical_transformer, categorical_features),
    ],
    remainder="passthrough",
)

# Define preprocessing pipeline (data transformation)
preprocessor = Pipeline(steps=[("data_transformer", data_transformer),])

# Define model pipeline
rf_model = RandomForestRegressor()
rfc_pipe = Pipeline(steps=[("preprocessor", preprocessor), ("regressor", rf_model)])

### Define function to run model and return metrics

In [156]:
def run_models(model_name, pipe, X_train, y_train, X_test, y_test):

    fit = pipe.fit(X_train, y_train)
    predictions = pipe.predict(X_test)
    score = pipe.score(X_test, y_test)

    print(len(y_test))
    print(len(predictions))

    print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, predictions))
    print("Mean Squared Error:", metrics.mean_squared_error(y_test, predictions))
    print(
        "Root Mean Squared Error:",
        np.sqrt(metrics.mean_squared_error(y_test, predictions)),
    )

    r2 = metrics.r2_score(y_test, predictions, multioutput="variance_weighted")
    n, p = X_test.shape

    print("R Square Score:", r2)

    ad_r_square = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

    print("Adjusted R Square Score:", ad_r_square)

    feats = {}  # a dict to hold feature_name: feature_importance
    for feature, importance in zip(
        X_test.columns, rfc_pipe.named_steps["regressor"].feature_importances_
    ):
        feats[feature] = importance  # add the name/value pair

    importances = pd.DataFrame.from_dict(feats, orient="index").rename(
        columns={0: "Gini-importance"}
    )
    print(importances.sort_values(by="Gini-importance", ascending=False))

    return y_test, predictions

### Define function to check predicted position accuracy

In [157]:
# Function to calculate euclidian distance between our predicted (x,y) points and the actual (x,y) points


def check_euclydian(y_test, predictions):
    val = []
    for i in range(len(y_test)):
        val.append(LA.norm(y_test.values[i] - preds[i]))

    print(np.mean(val))

### Run model

In [104]:
# Dictionary for testing on limited number of POI
dict1 = {
    "cam444_1": [X_train_444_1, y_train_444_1, X_test_444_1, y_test_444_1],
    "cam444_2": [X_train_444_2, y_train_444_2, X_test_444_2, y_test_444_2],
}

# Looping through all POI & predictive time window susbdataframes
for key, value in train_test_per_camera_dict.items():
    # for key, value in dict1.items():
    print("\n-----", key)
    y_test, preds = run_models("rfc", rfc_pipe, *value)
    check_euclydian(y_test, preds)


----- cam444_1


/Users/matt/opt/anaconda3/envs/propulsion/lib/python3.8/site-packages/sklearn/base.py:426: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  warnings.warn("The default value of multioutput (not exposed in "


29763
29763
Mean Absolute Error: 0.048450695585971475
Mean Squared Error: 0.02354204254228431
Root Mean Squared Error: 0.15343416354347003
R Square Score: 0.9961573387205479
Adjusted R Square Score: 0.9961548840063527
                    Gini-importance
y_world                    0.832300
x_world                    0.075077
y_velocity                 0.023895
looking_at_screen          0.020753
d_time                     0.014170
cumulative_time            0.013729
velocity                   0.005096
gender                     0.004203
x_velocity                 0.003668
dy_world                   0.002677
ema_5_velocity             0.002399
ema_5_x                    0.000891
distance_to_camera         0.000380
ema_10_x                   0.000365
ema_5_y                    0.000165
age                        0.000162
ema_10_velocity            0.000028
ema_10_y                   0.000025
dx_world                   0.000000
0.07874992241388705

----- cam450_1


/Users/matt/opt/anaconda3/envs/propulsion/lib/python3.8/site-packages/sklearn/base.py:426: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  warnings.warn("The default value of multioutput (not exposed in "


79876
79876
Mean Absolute Error: 0.029828544322200196
Mean Squared Error: 0.013840756789064271
Root Mean Squared Error: 0.11764674576487133
R Square Score: 0.9951776989591966
Adjusted R Square Score: 0.9951765515974482
                    Gini-importance
y_world                    0.729069
x_world                    0.140654
y_velocity                 0.036848
looking_at_screen          0.026956
cumulative_time            0.017770
d_time                     0.017650
velocity                   0.008353
gender                     0.006762
x_velocity                 0.004878
ema_5_velocity             0.004381
dy_world                   0.004154
distance_to_camera         0.000829
ema_5_x                    0.000807
ema_10_x                   0.000347
ema_5_y                    0.000250
age                        0.000199
ema_10_y                   0.000040
ema_10_velocity            0.000037
dx_world                   0.000000
0.048209091939393774

----- cam453_1


...

42733
42733
Mean Absolute Error: 0.9936763417792353
Mean Squared Error: 2.78907257135202
Root Mean Squared Error: 1.6700516672702135
R Square Score: 0.7362001267112773
Adjusted R Square Score: 0.7360827807605718
                    Gini-importance
x_world                    0.450162
y_velocity                 0.154222
y_world                    0.064712
ema_5_velocity             0.047480
dy_world                   0.047111
cumulative_time            0.029316
d_time                     0.027390
x_velocity                 0.027341
looking_at_screen          0.023794
gender                     0.021830
distance_to_camera         0.019329
velocity                   0.018531
age                        0.018296
ema_10_x                   0.015284
ema_5_y                    0.015093
ema_5_x                    0.014863
ema_10_y                   0.001879
ema_10_velocity            0.001787
dx_world                   0.000000
1.574085660825743

----- cam466_5


/Users/matt/opt/anaconda3/envs/propulsion/lib/python3.8/site-packages/sklearn/base.py:426: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  warnings.warn("The default value of multioutput (not exposed in "


79742
79742
Mean Absolute Error: 0.46288141430688967
Mean Squared Error: 0.8495427159797975
Root Mean Squared Error: 0.9217064152862328
R Square Score: 0.7421697167370886
Adjusted R Square Score: 0.7421082685122323
                    Gini-importance
y_velocity                 0.278115
y_world                    0.219502
x_world                    0.172055
cumulative_time            0.037398
d_time                     0.037380
gender                     0.034186
looking_at_screen          0.032811
x_velocity                 0.028862
dy_world                   0.027240
ema_5_velocity             0.025326
velocity                   0.018738
distance_to_camera         0.018142
age                        0.016877
ema_10_x                   0.015860
ema_5_y                    0.015833
ema_5_x                    0.015755
ema_10_y                   0.001920
ema_10_velocity            0.001853
dx_world                   0.000000
0.7389774521059894

----- cam529_5


/Users/matt/opt/anaconda3/envs/propulsion/lib/python3.8/site-packages/sklearn/base.py:426: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  warnings.warn("The default value of multioutput (not exposed in "


21565
21565
Mean Absolute Error: 0.959855425407284
Mean Squared Error: 4.279423819351628
Root Mean Squared Error: 2.0686768281564976
R Square Score: 0.5827778285161508
Adjusted R Square Score: 0.5824098906531574
                    Gini-importance
y_world                    0.314586
y_velocity                 0.109071
x_world                    0.105011
looking_at_screen          0.084513
d_time                     0.064330
cumulative_time            0.064052
distance_to_camera         0.031510
gender                     0.027520
age                        0.026893
velocity                   0.025972
ema_10_x                   0.025878
x_velocity                 0.025769
ema_5_y                    0.025084
ema_5_x                    0.023391
ema_5_velocity             0.019849
dy_world                   0.019506
ema_10_y                   0.002511
ema_10_velocity            0.002317
dx_world                   0.000000
1.6070671111325885


# END